<h1><center> Vigenér cipher decryption </center><h1>

---

In [63]:
import os
import sys

__module_path = os.path.abspath(os.path.join('..'))
if __module_path not in sys.path:
    sys.path.append(__module_path)

import cryptoanalysis

import numpy as np

from IPython.display import Javascript, HTML

from plotly import tools, graph_objs, __version__

# offline plotly
# import plotly.offline as plotly

# online plotly
import plotly.plotly as plotly

from ipywidgets import Layout, widgets, Box, HBox, VBox

In [64]:
%matplotlib inline

In [65]:
plotly_version = __version__

# For online connection:
# tools.set_credentials_file(username='CermakM', api_key='q2yoGYw052dyHMd8ztqx')
# tools.set_config_file(world_readable=False, sharing='public')

# For offline connection:
# plotly.init_notebook_mode(connected=True)

In [66]:
_submitted = False
_reset = True

cipher = ''
decrypted_text = ''

analyser = cryptoanalysis.decryption.Analyser(lang='en')

### Encrypted text

In [67]:
def _create_text_box(*args, **kwargs):
    for widget in args:
        widget.layout=Layout(width='100%', height='200px')
    box = Box(args, **kwargs)
    box.layout.display = 'flex'
    box.layout.align_items = 'stretch'
    box.layout.min_height = '230px'
    box.layout.flex_direction = 'row'
    return box

In [68]:
input_area = widgets.Textarea(
    placeholder='Encrypted text',
)  

output_area = widgets.Textarea(
    placeholder='Decrypted text',
    disabled=True
)

input_box = _create_text_box(input_area)
output_box = _create_text_box(output_area)

In [69]:
submit_input_button = widgets.Button(
    description='SUBMIT',
    disabled=False,
    button_style='success',
    icon='check'
)

reset_input_button = widgets.Button(
    description='RESET',
    disabled=False,
    button_style='danger'
)

In [70]:
def _on_submit(sender):
    global analyser, cipher, _submitted, _reset
            
    cipher = input_area.value
    
    if not cipher:
        return
    
    _reset = False

    analyser = cryptoanalysis.decryption.Analyser(cipher=cipher, lang='en')
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _submitted:
#         Javascript('$.notify("Cipher submitted");')
        
        _submitted = True
    
def _on_reset(sender):
    global analyser, cipher, _submitted
    
    _submitted = False
    
    cipher = ''
    analyser = cryptoanalysis.decryption.Analyser(lang='en')
    input_area.value = ''
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _reset:
#         Javascript('$.notify("Analyser has been reset")')
        
        _reset = True

In [71]:
submit_input_button.on_click(_on_submit)
reset_input_button.on_click(_on_reset)

In [72]:
submit_input_button.layout.margin = '2px 2px 2px auto'
reset_input_button.layout.margin = '2px 2px 2px 5px'

submit_box = HBox([submit_input_button, reset_input_button])

submit_box.layout.display = 'flex'
submit_box.layout.flex_flow = 'row'
submit_box.layout.align_items = 'flex-end'

In [73]:
button_samples = widgets.ToggleButtons(
    options=['Sample 1', 'Sample 2', 'Sample 3'],
    description='Load sample:',
    disabled=False,
    button_style='',
    tooltips=['Caesar cipher', 'Vigener cipher - short', 'Vigener cipher - long']
)

button_load = widgets.Button(
    description='LOAD',
    disabled=False,
    button_style='success',
    icon='upload'
)

samples = ['samples/TIKcipher1.txt', 'samples/TIKcipher2a.txt', 'samples/TIKcipher2b.txt']

def _on_sample_load(sender):
    sample_to_read = samples[button_samples.index]
    with open(os.path.join(__module_path, sample_to_read)) as s:
        input_area.value = s.read()
        
button_load.on_click(_on_sample_load)

In [74]:
def _on_input(sender):
    pass
    
input_area.observe(_on_input, names='value')

In [75]:
display(HBox([button_samples, button_load]))

A Jupyter Widget

In [76]:
display(VBox([input_box, submit_box]))

A Jupyter Widget

---
### Frequency Analysis

In [77]:
default_lang_trace = graph_objs.Bar(
    x=analyser.alphabet,
    y=analyser.letter_frequency,
    name='Language'
)

if cipher:
    cipher_dict = analyser.get_char_frequency()
    cipher_trace = graph_objs.Bar(
        x=list(cipher_dict.keys()),
        y=list(cipher_dict.values()),
        name='Cipher'
    )
    data = [default_lang_trace, cipher_trace]

else:
    data = [default_lang_trace]
    
layout = graph_objs.Layout(
    showlegend=True,
    title='Frequency analysis',
    xaxis=dict(tickangle=-45),
    yaxis=dict(
        title='Letter frequency',
        tickformat=' %'),
    barmode='group'
)


---

In [78]:
fig = graph_objs.Figure(data=data, layout=layout)

cipher_plot_url = plotly.iplot(fig, filename='cipher-plot', show_link=False, fileopt='overwrite')

In [79]:
cipher_plot_url

---

### Key decryption

In [80]:
widget_rotation = widgets.ToggleButtons(
    options=[1, 0],
    disabled=False,
    tooltips=['Rotation 1: `a` -> `b`', 'Rotation 0: `a` -> `a`']
)

options = [] if not cipher else analyser.get_key_len_list()

widget_key_len = widgets.ToggleButtons(
    options=['-'] if not options else options,
    disabled=not options,
    button_style=''
)

key = ''

In [81]:
def _on_len_change(sender):
    global key
    if _submitted:
        key, _ = analyser.get_keys(key_id=widget_key_len.index)
        
        display(Javascript('IPython.notebook.execute_cells_below()'))
    else:
        key = ''

widget_key_len.observe(_on_len_change, 'value')

_on_len_change(None)  # Run for initialization

In [82]:
display(widgets.Label(value="Suggested key length:"))
display(widget_key_len)

A Jupyter Widget

A Jupyter Widget

In [83]:
key_char_list = [c for c in key] or ['-']

In [84]:
toggle_button_layout = Layout(
    display='block',
    margin='0 auto',
    justify_content='center',
    min_width='200px',
)

button_up = widgets.Button(
    disabled=False,
    icon='angle-up',
    layout=toggle_button_layout,
    button_style='success'
)

button_down = widgets.Button(
    disabled=False,
    icon='angle-down',
    layout=toggle_button_layout,
    button_style='success'
)

button_key_layout = Layout(
    display=''
)

button_keys = [widgets.ToggleButton(description=k, tooltip=str(i)) for i, k in enumerate(key_char_list)]
button_keys_dict = dict((i, k) for i, k in enumerate(button_keys))

box_layout = Layout(
    overflow_x='auto',
    justify_content='center',
    width='80%',
    margin='0 auto',
)

key_box = Box(button_keys, layout=box_layout)
_custom_key = ''
_use_custom_key = False
_toggled_key_index = 0
_toggled_key_button = None

button_use_custom_key = widgets.Button(
    display='block',
    description='USE KEY',
    tooltip='Use custom key',
    disabled=False,
    button_style='success',
    icon='key',
)

button_reset_key = widgets.Button(
    display='block',
    description='RESET KEY',
    tooltip='Reset custom key',
    disabled=False,
    button_style='info',
    icon='key',
)

In [85]:
from itertools import cycle
from collections import deque

alphabet_deque = deque(analyser.alphabet)

# Dictionary of possible key variants
key_char_vectors = dict()
for index, char in enumerate(key_char_list):
    if char == '-':
        break
    key_char_vectors[index] = cycle(analyser.get_shift_vector(index))
    next(key_char_vectors[index])  # skip one at init

In [86]:
def _get_clicked_button(buttons: iter):
    for index, but in enumerate(buttons):
        if but.value is True:
            return index, but
    # Emergency case
    return 0

In [87]:
from collections import Counter

shift_letters = analyser.alphabet 
shift_freq = [0.0 for _ in analyser.alphabet]


def _get_shift_data():
    global shift_letters, shift_freq

    if _toggled_key_button is not None:

        key_len = len(key_char_list)
        # Get letters with stride key_len
        cipher_strip = cipher.replace(' ', '')[_toggled_key_index::key_len]

        # bag
        cipher_strip_bag = Counter(cipher_strip)
        # sort
        sorted_items = sorted([(k, v) for k, v in cipher_strip_bag.items()])
        # shift
        shift_strip = deque(sorted_items)
        shift_strip.rotate(ord(_toggled_key_button.description) - ord('a'))
        shift_letters = [t[0] for t in shift_strip]
        # freq
        shift_count = [t[1] for t in shift_strip]
        c_sum = sum(shift_count)
        shift_freq = [c / c_sum for c in shift_count]

    cipher_shift_trace = graph_objs.Bar(
        x=shift_letters,
        y=shift_freq,
        name='Shifted trace',
        marker=dict(
            color='orange',
        )
    )
    
    return cipher_shift_trace

In [88]:
def _on_toggle_up(sender):
    if not key_char_vectors:
        return
    index, button = _toggled_key_index, _toggled_key_button
    shift = int(next(key_char_vectors[index])) + 1
    
    alpha_d = alphabet_deque.copy()
    deque.rotate(alpha_d, shift)
    button.description = alpha_d[0]
    
def _on_toggle_down(sender):
    if not key_char_vectors:
        return
    index, button = _toggled_key_index, _toggled_key_button
    for i in range(4):  # 4 is magic (len of shift vector, stable on 5)
        shift = int(next(key_char_vectors[index])) + 1
        
    alpha_d = alphabet_deque.copy()
    deque.rotate(alpha_d, shift)
    
    button.description = alpha_d[0]
    
def _on_toggle_key(sender):
    global _toggled_key_index, _toggled_key_button, shift_plot_url, shift_fig
    if _toggled_key_button is not None:
        _toggled_key_button.value = False
    _toggled_key_index, _toggled_key_button = _get_clicked_button(key_box.children) 
    
    shift_plot_url = plotly.iplot(graph_objs.Data([_get_shift_data()]), filename='shift-plot')
    # HOW TO UPDATE?????
    
def _on_custom_key(sender):
    global _use_custom_key, _custom_key
    _use_custom_key = True
    _custom_key = "".join([b.description for b in key_box.children])
    output_area.value = analyser.decipher(custom_key=_custom_key, rot=widget_rotation.value)
    
def _on_reset_key(sender):
    global key, _custom_key, _use_custom_key
    _custom_key = key
    _use_custom_key = False
    for i, but in enumerate(key_box.children):
        but.description = key[i]
    output_area.value = analyser.decipher(custom_key=key, rot=widget_rotation.value)

In [89]:
button_up.on_click(_on_toggle_up)
button_down.on_click(_on_toggle_down)

button_use_custom_key.on_click(_on_custom_key)
button_reset_key.on_click(_on_reset_key)

for button in button_keys:
    button.observe(_on_toggle_key, names='value')

#### Decrypted key

In [90]:
button_custom_box = Box([button_use_custom_key, button_reset_key])
button_custom_box.layout.display = 'flex'
button_custom_box.layout.flex_flow = 'row'
button_custom_box.layout.justify_content = 'flex-end'

In [91]:
display(widgets.Label(value="Decrypted key:"))
display(button_up)
display(key_box)
display(button_down)
display(button_custom_box)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

### Corresponding shift

In [92]:
shift_layout = graph_objs.Layout(
    showlegend=True,
    title='Corresponding shift',
    xaxis=dict(tickangle=-45),
    yaxis=dict(
        title='Letter frequency',
        tickformat=' %')
)

In [93]:
shift_fig = graph_objs.Figure(data=[_get_shift_data()], layout=shift_layout)

shift_plot_url = plotly.iplot(shift_fig, filename='shift-plot', show_link=False, fileopt='overwrite')

In [94]:
display(shift_plot_url)

---
### Text decryption

In [95]:
def _on_rot_change(sender):
    use_key = _custom_key if _use_custom_key else key
    output_area.value = analyser.decipher(custom_key=use_key, rot=widget_rotation.value)

In [96]:
widget_rotation.observe(_on_rot_change, 'value')

---

In [97]:
display(widgets.Label(value="Select rotation:"))
display(widget_rotation)

A Jupyter Widget

A Jupyter Widget

In [98]:
result = ''
if _submitted:
    use_key = _custom_key if _use_custom_key else key
    result = analyser.decipher(custom_key=use_key, rot=widget_rotation.value)
    
output_area.value = result

In [99]:
display(widgets.Label(value="Decrypted text:"))
display(_create_text_box(output_area))

A Jupyter Widget

A Jupyter Widget